In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
import matplotlib.animation as animation
from IPython.display import HTML

In [ ]:
%matplotlib inline
GRID_SIZE = 100
DT = 1
DX = 1
DY = 1
time_steps = 10000

parameters = [
    {"DU": 0.16, "DV": 0.08, "f": 0.035, "k": 0.060},  # suggested
    {"DU": 0.14, "DV": 0.06, "f": 0.025, "k": 0.055},  # slightly lower DU, DV, f and k
    {"DU": 0.16, "DV": 0.08, "f": 0.025, "k": 0.050},  # same as suggested, but lower f and k
    {"DU": 0.18, "DV": 0.09, "f": 0.040, "k": 0.065},  # slightly higher DU, DV, f and k - diffuses very quickly
    {"DU": 0.16, "DV": 0.08, "f": 0.045, "k": 0.070},  # same as suggested, but higher f and k - diffuses very quickly
]

def initialize_grid():
    N = GRID_SIZE
    u0, v0 = 0.5, 0.25 # starting vals, u0 everywhere, v0 in the center
    U = np.ones((N, N)) * u0
    V = np.zeros((N, N))

    # square in V
    M = N // 11
    square_slice = slice(N // 2 - M, N // 2 + M)
    r = np.zeros(N)
    r[square_slice] = v0
    V[square_slice] = r

    # noise of 0.05 to both
    U += 0.05 * np.random.random((N, N))
    V += 0.05 * np.random.random((N, N))
    return U, V

@jit(nopython=True)
def update(U, V, DU, DV, f, k):
    N = GRID_SIZE
    U0 = U.copy()
    V0 = V.copy()

    for i in range(N):
        for j in range(N):
            ip = (i + 1) % N
            im = (i - 1) % N
            jp = (j + 1) % N
            jm = (j - 1) % N

            U[i, j] = U0[i, j] + DT * (DU / DX**2 * (U0[ip, j] + U0[im, j] + U0[i, jp] + U0[i, jm] - 4 * U0[i, j]) - U0[i, j] * V0[i, j]**2 + f * (1 - U0[i, j]))
            V[i, j] = V0[i, j] + DT * (DV / DX**2 * (V0[ip, j] + V0[im, j] + V0[i, jp] + V0[i, jm] - 4 * V0[i, j]) + U0[i, j] * V0[i, j]**2 - (f + k) * V0[i, j])

    return U, V

def plot(steps, DU, DV, f, k):
    U0, V0 = initialize_grid()
    time_intervals = [1000, 2500, 5000, 7500]
    font = 20
    
    fig, axes = plt.subplots(2, 4, figsize=(12, 5))
    u_min, u_max = 0, 1
    v_min, v_max = 0, 1  

    for col_idx, t in enumerate(time_intervals):
        U, V = initialize_grid()
        for _ in range(t):
            U, V = update(U, V, DU, DV, f, k)
        
        # U top row
        ax1 = axes[0, col_idx]
        ax1.imshow(U, cmap='inferno', vmin=u_min, vmax=u_max)
        ax1.set_title(f"U at t = {t}", fontsize=font)
        ax1.axis('off')

        # V bottom row
        ax2 = axes[1, col_idx]
        ax2.imshow(V, cmap='inferno', vmin=v_min, vmax=v_max)
        ax2.set_title(f"V at t = {t}", fontsize=font)
        ax2.axis('off')

    plt.subplots_adjust(wspace=0.05, hspace=0.02)
    plt.tight_layout(pad=0.5)
    plt.savefig(f"gs2_DU{DU}_DV{DV}_f{f}_k{k}.png", dpi=300)
    plt.show()




for parameter in parameters:
    print(f"Running simulation for DU={parameter['DU']}, DV={parameter['DV']}, f={parameter['f']}, k={parameter['k']}")
    plot(time_steps, **parameter)

In [ ]:
%matplotlib notebook

def animate_gray_scott(DU, DV, f, k, frames=200, updates_per_frame=5, interval=50):
    U, V = initialize_grid()
    
    fig, (ax_u, ax_v) = plt.subplots(1, 2, figsize=(10, 5))
    im_u = ax_u.imshow(U, cmap='inferno', vmin=0, vmax=1, animated=True)
    ax_u.set_title("U")
    im_v = ax_v.imshow(V, cmap='inferno', vmin=0, vmax=1, animated=True)
    ax_v.set_title("V")
    
    # remove axis
    ax_u.set_xticks([])
    ax_u.set_yticks([])
    ax_v.set_xticks([])
    ax_v.set_yticks([])
    
    def init():
        im_u.set_data(U)
        im_v.set_data(V)
        return [im_u, im_v]
    
    def animate(frame):
        nonlocal U, V
        for _ in range(updates_per_frame):
            U, V = update(U, V, DU, DV, f, k)
        im_u.set_data(U)
        im_v.set_data(V)
        return [im_u, im_v]
    
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=frames, interval=interval, blit=True)
    return anim


anim = animate_gray_scott(DU=0.18, DV=0.09, f=0.04, k=0.065, frames=1000, updates_per_frame=5, interval=100)
HTML(anim.to_jshtml())


In [ ]:
%matplotlib inline
def plot_avg_concentration_over_time(steps, record_interval, DU, DV, f, k):
    U, V = initialize_grid()
    avg_u = []
    avg_v = []
    times = []
    
    for t in range(steps):
        U, V = update(U, V, DU, DV, f, k)
        if t % record_interval == 0:
            times.append(t)
            avg_u.append(np.mean(U))
            avg_v.append(np.mean(V))
    
    plt.figure(figsize=(8, 4))
    plt.plot(times, avg_u, label="Average Concentration of U")
    plt.plot(times, avg_v, label="Average Concentration of V")
    plt.xlabel("Time Steps")
    plt.ylabel("Average Concentration")
    plt.title(f"Average Concentration Over Time (DU={DU}, DV={DV}, f={f}, k={k})")
    plt.legend()
    plt.grid(True)
    plt.show()

for parameter in parameters:
    plot_avg_concentration_over_time(steps=10000, record_interval=100, DU=parameter['DU'], DV=parameter['DV'], f=parameter['f'], k=parameter['k'])

